In [1]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

# authenticate
credential = DefaultAzureCredential()

SUBSCRIPTION="a3f56f48-3efb-4970-81a3-e4eda598333c"
RESOURCE_GROUP="luiz.victor.dev-rg"
WS_NAME="tcc-experiments"

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id=SUBSCRIPTION,
    resource_group_name=RESOURCE_GROUP,
    workspace_name=WS_NAME,
)

In [2]:
# Verify that the handle works correctly.  
# If you ge an error here, modify your SUBSCRIPTION, RESOURCE_GROUP, and WS_NAME in the previous cell.
ws = ml_client.workspaces.get(WS_NAME)
print(ws.location,":", ws.resource_group)

eastus2 : luiz.victor.dev-rg


In [3]:
TRAIN_DATAS = [
    "vrex_consolidated_2008_2017.csv"
]

TEST_DATAS = [
    "vrex_consolidated_2018_2021.csv"
]

version = "original"

arr_data_to_train = []
arr_data_to_test = []

for to_train, to_test in zip(TRAIN_DATAS, TEST_DATAS):
    data_to_train = ml_client.data.get(name=to_train.split(".")[0], version=version)
    arr_data_to_train.append(data_to_train)
    print(f"Data to train asset URI: {data_to_train.path} - name: {to_train.split('.')[0]}")

    data_to_test = ml_client.data.get(name=to_test.split('.')[0], version=version)
    arr_data_to_test.append(data_to_test)
    print(f"Data to test asset URI: {data_to_test.path} - name: {to_test.split('.')[0]}")

Data to train asset URI: azureml://subscriptions/a3f56f48-3efb-4970-81a3-e4eda598333c/resourcegroups/luiz.victor.dev-rg/workspaces/tcc-experiments/datastores/workspaceblobstore/paths/LocalUpload/4722af391476dd8597905df3c3ae6d19/vrex_consolidated_2008_2017.csv - name: vrex_consolidated_2008_2017
Data to test asset URI: azureml://subscriptions/a3f56f48-3efb-4970-81a3-e4eda598333c/resourcegroups/luiz.victor.dev-rg/workspaces/tcc-experiments/datastores/workspaceblobstore/paths/LocalUpload/154dc9171c762e1cab20b8998d7d1c63/vrex_consolidated_2018_2021.csv - name: vrex_consolidated_2018_2021


In [4]:
# importing the Component Package
from azure.ai.ml import load_component

# Loading the component from the yml file
data_prep_component = load_component(source="/home/azureuser/cloudfiles/code/Users/luiz.victor.dev/tcc_experiments_az_ml/src/components/preparation/data_prep.yaml")

# Now we register the component to the workspace
data_prep_component = ml_client.create_or_update(data_prep_component)

# Create (register) the component in your workspace
print(
    f"Component {data_prep_component.name} with Version {data_prep_component.version} is registered"
)

Component data_prep_vrex_defaults_model with Version 2024-05-08-23-43-30-2231322 is registered


In [5]:
# importing the Component Package
from azure.ai.ml import load_component

# Loading the component from the yml file
train_gbc = load_component(source="/home/azureuser/cloudfiles/code/Users/luiz.victor.dev/tcc_experiments_az_ml/src/components/train/train_gbc.yaml")

# Now we register the component to the workspace
train_gbc = ml_client.create_or_update(train_gbc)

# Create (register) the component in your workspace
print(
    f"Component {train_gbc.name} with Version {train_gbc.version} is registered"
)

Component train_gbc_model with Version 2024-05-08-23-43-32-1664066 is registered


In [6]:
# importing the Component Package
from azure.ai.ml import load_component

# Loading the component from the yml file
train_nbc = load_component(source="/home/azureuser/cloudfiles/code/Users/luiz.victor.dev/tcc_experiments_az_ml/src/components/train/train_nbc.yaml")

# Now we register the component to the workspace
train_nbc = ml_client.create_or_update(train_nbc)

# Create (register) the component in your workspace
print(
    f"Component {train_nbc.name} with Version {train_nbc.version} is registered"
)

Component train_nbc_model with Version 2024-05-08-23-43-33-6218937 is registered


In [7]:
# importing the Component Package
from azure.ai.ml import load_component

# Loading the component from the yml file
train_rfc = load_component(source="/home/azureuser/cloudfiles/code/Users/luiz.victor.dev/tcc_experiments_az_ml/src/components/train/train_rfc.yaml")

# Now we register the component to the workspace
train_rfc = ml_client.create_or_update(train_rfc)

# Create (register) the component in your workspace
print(
    f"Component {train_rfc.name} with Version {train_rfc.version} is registered"
)

Component train_rfc_model with Version 2024-05-08-23-43-35-1402885 is registered


In [8]:
# importing the Component Package
from azure.ai.ml import load_component

# Loading the component from the yml file
train_svc = load_component(source="/home/azureuser/cloudfiles/code/Users/luiz.victor.dev/tcc_experiments_az_ml/src/components/train/train_svc.yaml")

# Now we register the component to the workspace
train_svc = ml_client.create_or_update(train_svc)

# Create (register) the component in your workspace
print(
    f"Component {train_svc.name} with Version {train_svc.version} is registered"
)

Component train_svc_model with Version 2024-05-08-23-43-36-5992663 is registered


In [9]:
# importing the Component Package
from azure.ai.ml import load_component

# Loading the component from the yml file
train_xgb = load_component(source="/home/azureuser/cloudfiles/code/Users/luiz.victor.dev/tcc_experiments_az_ml/src/components/train/train_xgb.yaml")

# Now we register the component to the workspace
train_xgb = ml_client.create_or_update(train_xgb)

# Create (register) the component in your workspace
print(
    f"Component {train_xgb.name} with Version {train_xgb.version} is registered"
)

Component train_xgb_model with Version 2024-05-08-23-43-38-2536869 is registered


In [10]:
TOTAL_TRIALS = 20
CONCURRENT_TRIALS = 10
TIMEOUT = 3600
TIMEOUT_PLUS = 7200

COMPUTE = 'serverless'
SAMPLING_ALGORITHM = 'bayesian'
METRIC = 'training_accuracy_score'
GOAL = 'Maximize'
EVALUATION_INTERVAL = 1
DELAY_EVALUATION = 5

# the dsl decorator tells the sdk that we are defining an Azure Machine Learning pipeline
from azure.ai.ml import dsl, Input, Output
from azure.ai.ml.sweep import Choice, Uniform, MedianStoppingPolicy

In [11]:
@dsl.pipeline(
    compute="serverless",
    description="E2E data_perp-train pipeline",
)
def train_gbc_pipeline(
    data_to_train,
    data_to_test,
    flag_remove_null_values,
    flag_remove_values_by_percentage,
    percentage_to_remove_column,
):

    data_prep_job = data_prep_component(
        data_to_train=data_to_train,
        data_to_test=data_to_test,
        flag_remove_null_values=flag_remove_null_values,
        flag_remove_values_by_percentage=flag_remove_values_by_percentage,
        percentage_to_remove_column=percentage_to_remove_column,
    )

    train_gbc_job = train_gbc(
        train_data=data_prep_job.outputs.train_data,  # note: using outputs from previous step
        test_data=data_prep_job.outputs.test_data,  # note: using outputs from previous step
        n_estimators_to_gbc=Choice(values=[50, 100, 200]),
        learning_rate_to_gbc=Uniform(min_value=0.01, max_value=0.3),
    )

    sweep_step_to_gbc = train_gbc_job.sweep(
        compute=COMPUTE,
        sampling_algorithm=SAMPLING_ALGORITHM,
        primary_metric=METRIC,
        goal=GOAL,
    )

    sweep_step_to_gbc.set_limits(max_total_trials=2*TOTAL_TRIALS, max_concurrent_trials=CONCURRENT_TRIALS, timeout=TIMEOUT)

    sweep_step_to_gbc.early_termination = MedianStoppingPolicy(delay_evaluation=DELAY_EVALUATION, evaluation_interval=EVALUATION_INTERVAL)


In [12]:
@dsl.pipeline(
    compute="serverless",
    description="E2E data_perp-train pipeline",
)
def train_nbc_pipeline(
    data_to_train,
    data_to_test,
    flag_remove_null_values,
    flag_remove_values_by_percentage,
    percentage_to_remove_column,
):

    data_prep_job = data_prep_component(
        data_to_train=data_to_train,
        data_to_test=data_to_test,
        flag_remove_null_values=flag_remove_null_values,
        flag_remove_values_by_percentage=flag_remove_values_by_percentage,
        percentage_to_remove_column=percentage_to_remove_column,
    )

    train_nbc_job = train_nbc(
        train_data=data_prep_job.outputs.train_data,  # note: using outputs from previous step
        test_data=data_prep_job.outputs.test_data,  # note: using outputs from previous step
    )

    sweep_step_to_nbc = train_nbc_job.sweep(
        compute=COMPUTE,
        sampling_algorithm=SAMPLING_ALGORITHM,
        primary_metric=METRIC,
        goal=GOAL,
    )

    sweep_step_to_nbc.set_limits(max_total_trials=1, max_concurrent_trials=1, timeout=TIMEOUT)
    sweep_step_to_nbc.early_termination = MedianStoppingPolicy(delay_evaluation=DELAY_EVALUATION, evaluation_interval=EVALUATION_INTERVAL)


In [13]:
@dsl.pipeline(
    compute="serverless",
    description="E2E data_perp-train pipeline",
)
def train_rfc_pipeline(
    data_to_train,
    data_to_test,
    flag_remove_null_values,
    learning_rate_to_train,
    flag_remove_values_by_percentage,
    percentage_to_remove_column,
):

    data_prep_job = data_prep_component(
        data_to_train=data_to_train,
        data_to_test=data_to_test,
        flag_remove_null_values=flag_remove_null_values,
        flag_remove_values_by_percentage=flag_remove_values_by_percentage,
        percentage_to_remove_column=percentage_to_remove_column,
    )

    train_rfc_job = train_rfc(
        train_data=data_prep_job.outputs.train_data,  # note: using outputs from previous step
        test_data=data_prep_job.outputs.test_data,  # note: using outputs from previous step
        n_estimators_to_rfc=Choice(values=[50, 100, 200]),
    )

    sweep_step_to_rfc = train_rfc_job.sweep(
        compute=COMPUTE,
        sampling_algorithm=SAMPLING_ALGORITHM,
        primary_metric=METRIC,
        goal=GOAL,
    )

    sweep_step_to_rfc.set_limits(max_total_trials=TOTAL_TRIALS, max_concurrent_trials=CONCURRENT_TRIALS, timeout=TIMEOUT)
    sweep_step_to_rfc.early_termination = MedianStoppingPolicy(delay_evaluation=DELAY_EVALUATION, evaluation_interval=EVALUATION_INTERVAL)

In [14]:
@dsl.pipeline(
    compute="serverless",
    description="E2E data_perp-train pipeline",
)
def train_svc_pipeline(
    data_to_train,
    data_to_test,
    flag_remove_null_values,
    learning_rate_to_train,
    flag_remove_values_by_percentage,
    percentage_to_remove_column,
):

    data_prep_job = data_prep_component(
        data_to_train=data_to_train,
        data_to_test=data_to_test,
        flag_remove_null_values=flag_remove_null_values,
        flag_remove_values_by_percentage=flag_remove_values_by_percentage,
        percentage_to_remove_column=percentage_to_remove_column,
    )

    train_svc_job = train_svc(
        train_data=data_prep_job.outputs.train_data,  # note: using outputs from previous step
        test_data=data_prep_job.outputs.test_data,  # note: using outputs from previous step
        kernel_to_svc=Choice(values=["linear", "rbf", "poly", "sigmoid","precomputed"]),
        gamma_to_svc=Choice(values=["scale", "auto"]),
    )

    sweep_step_to_svc = train_svc_job.sweep(
        compute=COMPUTE,
        sampling_algorithm=SAMPLING_ALGORITHM,
        primary_metric=METRIC,
        goal=GOAL,
    )

    sweep_step_to_svc.set_limits(max_total_trials=2*TOTAL_TRIALS, max_concurrent_trials=CONCURRENT_TRIALS, timeout=TIMEOUT_PLUS)
    sweep_step_to_svc.early_termination = MedianStoppingPolicy(delay_evaluation=DELAY_EVALUATION, evaluation_interval=EVALUATION_INTERVAL)

In [15]:
@dsl.pipeline(
    compute="serverless",
    description="E2E data_perp-train pipeline",
)
def train_xgb_pipeline(
    data_to_train,
    data_to_test,
    flag_remove_null_values,
    learning_rate_to_train,
    flag_remove_values_by_percentage,
    percentage_to_remove_column,
):

    data_prep_job = data_prep_component(
        data_to_train=data_to_train,
        data_to_test=data_to_test,
        flag_remove_null_values=flag_remove_null_values,
        flag_remove_values_by_percentage=flag_remove_values_by_percentage,
        percentage_to_remove_column=percentage_to_remove_column,
    )

    train_xgb_job = train_xgb(
        train_data=data_prep_job.outputs.train_data,  # note: using outputs from previous step
        test_data=data_prep_job.outputs.test_data,  # note: using outputs from previous step
        n_estimators_to_xgb=Choice(values=[100, 500, 1000]),
        learning_rate_to_xgb=Uniform(min_value=0.01, max_value=0.3),
    )

    sweep_step_to_xgb = train_xgb_job.sweep(
        compute=COMPUTE,
        sampling_algorithm=SAMPLING_ALGORITHM,
        primary_metric=METRIC,
        goal=GOAL,
    )

    sweep_step_to_xgb.set_limits(max_total_trials=2*TOTAL_TRIALS, max_concurrent_trials=CONCURRENT_TRIALS, timeout=TIMEOUT)
    sweep_step_to_xgb.early_termination = MedianStoppingPolicy(delay_evaluation=DELAY_EVALUATION, evaluation_interval=EVALUATION_INTERVAL)

In [16]:
pipelines = []

for data_to_train, data_to_test in zip(arr_data_to_train, arr_data_to_test):

    #pipelines.append(train_gbc_pipeline(
    #    data_to_train=Input(type="uri_file", path=data_to_train.path),
    #    data_to_test=Input(type="uri_file", path=data_to_test.path),
    #    flag_remove_null_values=False,
    #    flag_remove_values_by_percentage=False,
    #    percentage_to_remove_column=0,
    #))

    #pipelines.append(train_nbc_pipeline(
    #    data_to_train=Input(type="uri_file", path=data_to_train.path),
    #    data_to_test=Input(type="uri_file", path=data_to_test.path),
    #    flag_remove_null_values=False,
    #    flag_remove_values_by_percentage=False,
    #    percentage_to_remove_column=0,
    #))

    #pipelines.append(train_rfc_pipeline(
    #    data_to_train=Input(type="uri_file", path=data_to_train.path),
    #    data_to_test=Input(type="uri_file", path=data_to_test.path),
    #    flag_remove_null_values=False,
    #    flag_remove_values_by_percentage=False,
    #    percentage_to_remove_column=0,
    #))

    pipelines.append(train_svc_pipeline(
        data_to_train=Input(type="uri_file", path=data_to_train.path),
        data_to_test=Input(type="uri_file", path=data_to_test.path),
        flag_remove_null_values=False,
        flag_remove_values_by_percentage=False,
        percentage_to_remove_column=0,
    ))

    pipelines.append(train_xgb_pipeline(
        data_to_train=Input(type="uri_file", path=data_to_train.path),
        data_to_test=Input(type="uri_file", path=data_to_test.path),
        flag_remove_null_values=False,
        flag_remove_values_by_percentage=False,
        percentage_to_remove_column=0,
    ))

print(pipelines)

[PipelineJob({'inputs': {'data_to_train': <azure.ai.ml.entities._job.pipeline._io.base.PipelineInput object at 0x7f71304cc640>, 'data_to_test': <azure.ai.ml.entities._job.pipeline._io.base.PipelineInput object at 0x7f71304cc1f0>, 'flag_remove_null_values': <azure.ai.ml.entities._job.pipeline._io.base.PipelineInput object at 0x7f71304cc4c0>, 'flag_remove_values_by_percentage': <azure.ai.ml.entities._job.pipeline._io.base.PipelineInput object at 0x7f71304cc280>, 'percentage_to_remove_column': <azure.ai.ml.entities._job.pipeline._io.base.PipelineInput object at 0x7f71304ccb20>}, 'outputs': {}, 'jobs': {}, 'component': PipelineComponent({'intellectual_property': None, 'auto_increment_version': True, 'source': 'DSL', 'is_anonymous': False, 'auto_delete_setting': None, 'name': 'train_rfc_pipeline', 'description': 'E2E data_perp-train pipeline', 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': None, 'Resource__source_path': 'None', 'base_path': '/mnt/batch/tasks/shared/LS_root/mount

In [17]:
import datetime as dt

models = ['svc', 'xgb']

def _get_experiment_names() -> [str]:
    experiment_names = []
    for train_name, test_name in zip(TRAIN_DATAS, TEST_DATAS):
        for model_name in models:
            current_time = dt.datetime.now()
            formatted_time = current_time.strftime("%Y_%m_%d_%H_%M_%S")  # Formata a data e hora atual
            train_name_base = train_name.split('.')[0]
            test_name_base = test_name.split('.')[0]
            name = f"{train_name_base}_tested_{test_name_base}_model_{model_name}"
            experiment_names.append(name)
            print(name)
    return experiment_names


In [18]:
experiment_names = _get_experiment_names()
for pipeline, experiment_name in zip(pipelines, experiment_names):
    print(experiment_name)
    print(pipeline)
    pipeline_job = ml_client.jobs.create_or_update(
        pipeline,
        experiment_name=experiment_name,
    )

    ml_client.jobs.stream(pipeline_job.name)

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


vrex_consolidated_2008_2017_tested_vrex_consolidated_2018_2021_model_rfc
vrex_consolidated_2008_2017_tested_vrex_consolidated_2018_2021_model_svc
vrex_consolidated_2008_2017_tested_vrex_consolidated_2018_2021_model_xgb
vrex_consolidated_2008_2017_tested_vrex_consolidated_2018_2021_model_rfc
display_name: train_rfc_pipeline
description: E2E data_perp-train pipeline
type: pipeline
inputs:
  data_to_train:
    type: uri_file
    path: azureml://subscriptions/a3f56f48-3efb-4970-81a3-e4eda598333c/resourcegroups/luiz.victor.dev-rg/workspaces/tcc-experiments/datastores/workspaceblobstore/paths/LocalUpload/4722af391476dd8597905df3c3ae6d19/vrex_consolidated_2008_2017.csv
  data_to_test:
    type: uri_file
    path: azureml://subscriptions/a3f56f48-3efb-4970-81a3-e4eda598333c/resourcegroups/luiz.victor.dev-rg/workspaces/tcc-experiments/datastores/workspaceblobstore/paths/LocalUpload/154dc9171c762e1cab20b8998d7d1c63/vrex_consolidated_2018_2021.csv
  flag_remove_null_values: 0
  flag_remove_values